In [9]:

import sys

sys.path.append(
    "C:\\Users\\lauth\\OneDrive\\Desktop\\LLM Assistant File - V2\\"
)
sys.path.append(
    "C:\\Users\\lauth\\OneDrive\\Desktop\\LLM Assistant File - V2\\assistant\\sql_assistant_v3\\"
)

from assistant.sql_assistant_v3.src.components.tools.tools import action
from assistant.sql_assistant_v3.src.app.notebooks.multi_agent.base import ChatMessage, ChatTemplate

# Supervisor

In [10]:
@action
def chat_sql_expert(message: str, **kwargs) -> str:
    """
    To chat with an SQL expert AI to generate or refine SQL queries based on the user's needs.

    Args:
        message: A message that includes a detailed description of the current user task.
        tables: Comma separated list of tables.
    """
agents = {
    "chat_sql_expert": chat_sql_expert,
}

In [11]:
supervisor_prompt_3 = """You are a helpful assistant who responds to user's request in a fun, friendly but profesional way. 

You will be leading a team of AI experts to solve any to fulfill user requests.

To communicate with your team, you have been given access to this actions: {{agent_experts}}, user_calling.

The way you use the actions is by specifying a json blob, ending with '<end_action>'.
Specifically, this json should have an `action` key (name of the action to use) and an `action_input` key (input to the action).

The $ACTION_JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. It should be formatted in json. Do not try to escape special characters. Here is the template of a valid $ACTION_JSON_BLOB:
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}<end_action>

Make sure to have the $INPUT as a dictionary in the right format for the action you are using, and do not put variable names as input if you can find the right values.

You should ALWAYS use the following format:

Thought: you should always think about **ONE ACTION** to take. Then use the action as follows:
Action:
$ACTION_JSON_BLOB
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times, you should take several steps when needed. The $ACTION_JSON_BLOB must only use a SINGLE action at a time.)

You can use the result of the previous action as input for the next action.
The observation will always be a string: it can represent a file, like "image_1.jpg".
Then you can use it as input for the next action. You can do it for instance as follows:

Observation: "image_1.jpg"

Thought: I need to know what is the image that I received about in the previous observation, let's chat with the images expert.
Action:
{
  "action": "image_expert",
  "action_input": {"message": "I have an image here, ¿can you describe this image?", "image": "image_1.jpg"}
}<end_action>


To provide the final answer to your boss, use an action blob with "action": "user_calling". It is the only way you can talk with your boss, else you will be stuck on a loop. So your final output should look like this:
Action:
{
  "action": "user_calling",
  "action_input": {"answer": "insert your final answer here"}
}<end_action>


You only have access to communicate with this experts:
{{agents}}

- user_calling: To communicate any inquiry or answer to the user.
    Takes inputs: {'answer': {'type': 'string', 'description': 'A friendly message with the final answer or inquiry for the user.'}}
    Returns an output of type: any

Here are the rules you should always follow to solve your task:
1. ALWAYS provide a 'Thought:' sequence, and an 'Action:' sequence that ends with <end_action>, else you will fail.
2. Always use the right arguments for the actions. Never use variable names in the 'action_input' field, use the value instead.
3. Never re-do an action call that you previously did with the exact same parameters.
4. Do not perform an action if the user hasn’t provided all required parameters. Instead, politely ask for the missing information.
5. Your are not allowed to answer with many actions, **only one**.
4. Ensure your responses remain fun, friendly, and professional, maintaining a tone suitable for the context.

Now Begin! If you solve the task correctly, you will receive a reward of $1,000,000."""

def get_task_prompt_3(
    agents,
):
    mask = """- {name}: {description}
    Takes inputs: {inputs}
    Returns an output of type: {output_type}"""

    agents_descriptions = "\n\n".join(
        [
            mask.format(
                name=agents[t].name,
                description=agents[t].description,
                inputs=agents[t].inputs,
                output_type=agents[t].output_type,
            )
            for t in agents
        ]
    )

    agents_experts = f"""{", ".join([agents[t].name for t in agents])}"""
    content = (
        supervisor_prompt_3.replace("{{agents}}", agents_descriptions)
        .replace("{{agent_experts}}", agents_experts)
    )

    return content

# SQL AGENTE

In [12]:
coder_agent_prompt = """You are an SQL SERVER 2014 expert analyst who can create any SQL SERVER 2014 query code for every task that your supervisor require.
By using JSON tool calls, you can retrieve information from database. 
You will be given a task by your supervisor and you have to solve as best as you can.

To do so, you have been given access to the following tools: {{tool_names}}, supervisor_call.
The way you use the tools is by specifying a json blob, ending with '<end_action>'.
Specifically, this json should have an `action` key (name of the tool to use) and an `action_input` key (input to the tool).

The $ACTION_JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. It should be formatted in json. Do not try to escape special characters. Here is the template of a valid $ACTION_JSON_BLOB:
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}<end_action>

Make sure to have the $INPUT as a dictionary in the right format for the tool you are using, and do not put variable names as input if you can find the right values.

You should ALWAYS use the following format:

Thought: you should always think about **ONE ACTION** to take. Then use the action as follows:
Action:
$ACTION_JSON_BLOB
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times, you should take several steps when needed. The $ACTION_JSON_BLOB must only use a SINGLE action at a time.)

You can use the result of the previous action as input for the next action.
The observation will always be a string: it can represent a file, like "image_1.jpg".
Then you can use it as input for the next action. You can do it for instance as follows:

Observation: "image_1.jpg"

Thought: I need to transform the image that I received in the previous observation to make it green.
Action:
{
  "action": "image_transformer",
  "action_input": {"image": "image_1.jpg"}
}<end_action>


To provide the final answer to the task, use an action blob with "action": "supervisor_call". It is the only way to complete the task and call your supervisor, else you will be stuck on a loop. So your final output should look like this:
Action:
{
  "action": "supervisor_call",
  "action_input": {"answer": "insert your final answer here"}
}<end_action>


You only have acces to those tools:

{{tools}}

- supervisor_call: To communicate any inquiry or answer to your supervisor.
    Takes inputs: {'answer': {'type': 'any', 'description': 'A message to your supervisor, inquiry or final answer.'}}
    Returns an output of type: any

Here are the rules you should always follow to solve your task:
1. ALWAYS provide a 'Thought:' sequence, and an 'Action:' sequence that ends with <end_action>, else you will fail.
2. Always use the right arguments for the tools. Never use variable names in the 'action_input' field, use the value instead.
3. Never re-do a tool call that you previously did with the exact same parameters.
4. Your are not allowed to answer with many action calls, **ONLY ONE**.

Now Begin! If you solve the task correctly, you will receive a reward of $1,000,000."""

def get_coder_agent_prompt(
    ts,
):
    mask = """- {name}: {description}
    Takes inputs: {inputs}
    Returns an output of type: {output_type}"""

    tools_descriptions = "\n\n".join(
        [
            mask.format(
                name=ts[t].name,
                description=ts[t].description,
                inputs=ts[t].inputs,
                output_type=ts[t].output_type,
            )
            for t in ts
        ]
    )

    tool_names = f"""{", ".join([ts[t].name for t in ts])}"""
    content = (
        coder_agent_prompt.replace("{{tool_names}}", tool_names)
        .replace("{{tools}}", tools_descriptions)
    )

    return content


In [19]:
from assistant.sql_assistant_v3.src.app.notebooks.multi_agent.tools import (
    retrieve_db_info,
    retrieve_table_schemas,
    sql_executor,
)

tools_3 = {
    "retrieve_db_info": retrieve_db_info,
    "retrieve_table_schemas": retrieve_table_schemas,
    "sql_executor": sql_executor,
}

system = get_coder_agent_prompt(tools_3)

# Preguntas

In [57]:
from assistant.sql_assistant_v3.src.app.notebooks.multi_agent.base import (
    chat_stream_response,
    handle_agent_response,
)


supervisor_system_prompt = get_task_prompt_3(agents)
sql_expert_system_prompt = get_coder_agent_prompt(tools_3)


def generar_preguntas(supervisor_llm, sql_llm, request_usuario):
    messages = [
        ChatMessage(role="system", content=supervisor_system_prompt),
        ChatMessage(
            role="user",
            content=f"""User:\n{request_usuario}""",
        ),
    ]

    ct = ChatTemplate(messages)

    r, _, u = chat_stream_response(
        llm=supervisor_llm, chat_messages=ct, has_stream=True
    )
    parsed_response = handle_agent_response(r, u)

    message_generated_by_llm = parsed_response.parameters["message"].strip()

    messages_ = [
        ChatMessage(
            role="system",
            content=sql_expert_system_prompt,
        ),
        ChatMessage(
            role="user",
            content=f"""Supervisor: {message_generated_by_llm}
        
Make sure you to follow this plan.
1. Use keywords and subquerys to find contextual info and some tables.
2. Retrieves the schema of the tables best related to the requirement.
3. Create the code.

Here is the criteria you have to follow to create valid syntactically SQL code:
1. Tables and columns aimed: Ensure that the columns and tables exists in Database information and are selected correctly because this are necessary and can respond correctly the main request. Never show id's in your final answer. You are not allowed to select all columns, else, db will return an extensive answer.
2. Proper Use of JOINs: Correct JOIN Type: Ensure that the JOIN type (e.g., INNER JOIN, LEFT JOIN, RIGHT JOIN) is appropriate for the data needed.
3. Sorting: Always use an ORDER BY statement, that will reflects the priority of the request. 
4. Aliases: Always use alias names for each table in your query or subquery.
5. Schema: Make sure your code is querying tables in the schema: 'dbo_v2'.""",
        ),
    ]
    ct_ = ChatTemplate(messages_)
    res, _, _ = chat_stream_response(llm=sql_llm, chat_messages=ct_, has_stream=True)
    
    parsed_response_ = handle_agent_response(res, u)
    
    return parsed_response_.parameters["keywords"].strip(), parsed_response_.parameters["sub_queries"]

In [ ]:
from openpyxl import Workbook, load_workbook

# Cargar el archivo Excel existente o crear uno nuevo si no existe
def _save_excel(data: list, sheet_name:str):
    archivo_excel = r"C:\Users\lauth\OneDrive\Desktop\LLM Assistant File - V2\fine_tuning\embeddings\sheets\datasets.xlsx"
    
    try:
        # Intenta cargar el archivo existente
        libro = load_workbook(archivo_excel)
    except FileNotFoundError:
        # Si el archivo no existe, crea uno nuevo
        libro = Workbook()

    # Selecciona la hoja llamada "nombres" o créala si no existe
    if sheet_name in libro.sheetnames:
        hoja = libro[sheet_name]
    else:
        hoja = libro.create_sheet(sheet_name)

    # Encuentra la primera fila vacía en la columna A
    fila_vacia = hoja.max_row + 1

    # Agrega los nuevos nombres a la hoja
    for i, nombre in enumerate(data, start=fila_vacia):
        hoja.cell(row=i, column=1, value=nombre)

    # Guarda los cambios en el archivo Excel
    libro.save(archivo_excel)

    print(f"Se han agregado {len(data)} {sheet_name} al archivo.")

In [ ]:
from assistant.sql_assistant_v3.src.app.notebooks.multi_agent.base import (
    llama_70b_hf,
    qwen_coder,
    openai_llm,
    qwen_instruct,
)


request = "latest uncertainty value registered for the differential pressure in the system with **tag** EMED-3138.12-050"
request = "the calibration points for the calibration with number: LMH 3305/2021"
request = "the calibration points for the equipment with tag: TE-EMED-3138.12-050"
request = "list the tags of the equipments with its serials and the name of the equipment type associated and also its equipment classification to the system with tag: EMED-3138.11-128"
request = "list the secondary equipments class, serials, equipment tag and the name of the equipment type linked to the system with tag: EMED-3138.11-128"
request = "when is the next calibration for the equipment with tag EMED-3138.12-015-DPT"
request = "list of the equipments serials associated to the system with tag EMED-3138.11-128. Also with the variable of each equipment can read"
request = "the last uncertainty report for temperature in the equipment with tag EMED-3138.12-050-TT"
request = "the lastest uncertainty reports for the temperature and differential pressure measured in the system with tag EMED-3138.12-050"
request = "the last uncertainty report for differential pressure in the equipment with tag EMED-3138.12-050-DPT"
request = "the last uncertainty report for temperature in the system with tag EMED-3138.12-050"
request = "the calibration points for the calibration with number: LMH 5643/2023"
request = "list the tags of the flow computers and the quantity of meter streams for each one"
request = "list the system tag, computer tag from flow computers that are associated with a measurement system and are flowboss"
request = "list the system tag, computer tag and the computer type of flow computers that are associated with a measurement system"
request = "list the system tag, computer tag from fc-302 computers that are associated with a measurement system"
request  = "Report for the computer tag: FQI-EMED-3138.12-050 at august 10th" # computador con id 12
request  = "The user wants a daily Report for the computer tag: FQI-EMED-3138.12-050 at august 10th 2023" # computador con id 12
request  = "The user wants a daily Report for the computer tag: FQI-EMED-3138.12-050 at august 2023" # computador con id 12
request  = "The user wants the the date of the lastest report for the computer tag: FQI-EMED-3138.12-050" # computador con id 12
request  = "The user wants a daily Report for the computer tag: FQI-EMED-3138.12-050 at 2023-08-10" # computador con id 12
request  = "the most recent uncertainty result for the specific system with tag EMED-3102-02-010"
request  = "the uncertainty result for the specific measurement system tagged as EMED-3102-02-010 at July 28th in 2023"
request  = "the flow rate uncertainty fingerprint points for the curve of uncertainty with certificate number EMED-010-1"
request  = "the initial conditions of static pressure, differential pressure, temperature and flow rate volume for uncertainty of the measurement system with tag EMED-3102-02-010 at July 28th in 2023"


kw, sub_queries = generar_preguntas(qwen_coder, openai_llm, request)

kw_list = [item.strip() for item in kw.split(", ")]

# _save_excel(kw_list, "keywords")
# _save_excel(sub_queries, "sub_queries")

In [96]:
import pandas as pd


def read_sheet(sheet_name: str  = "", cols: list[str] = None, *, path = None):
    """To read an excel that contains views definitions"""
    if cols is not None:
        ex_df = pd.read_excel(io=path, sheet_name=sheet_name, usecols=cols)
    else: 
        ex_df = pd.read_excel(io=path, sheet_name=sheet_name)
    return ex_df




In [99]:
path = r"C:\Users\lauth\OneDrive\Desktop\LLM Assistant File - V2\fine_tuning\embeddings\sheets\datasets.xlsx"

chunks_vals = read_sheet(sheet_name="chunks", cols=["id", "topic"], path=path)
excel_queries = read_sheet(sheet_name="sub_queries", cols=["question", "chunk"], path=path)

In [127]:
chunks_vals_lst = [f"""{chunk["id"]} -- {chunk["topic"]}""" for _, chunk in chunks_vals.iterrows()]
excel_queries_copy = excel_queries.copy()


for idx, item in excel_queries_copy.iterrows():
    question = item["question"]
    asd = "\n".join(chunks_vals_lst)
    txt = f"""{question}\n\n{asd}"""
    print(asd)
    
    chunk_id = input(f"### {question} ###") or None
    
    if not chunk_id:
        break
    else:
        val = f"c{chunk_id}"
        excel_queries_copy["chunk"][idx] = val
        print(val)

c0 -- flow computers
c1 -- reportes
c2 -- meter streams
c3 -- measurement system
c4 -- equipments/equipment tags
c5 -- equipment types, equipment class
c6 -- variables
c7 -- calibraciones
c8 -- incerteza


C:\Users\lauth\AppData\Local\Temp\ipykernel_16892\3870049061.py:17: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  excel_queries_copy["chunk"][idx] = val
C:\Users\lauth\AppData\Local\Temp\ipykernel_16892\3870049061.py:17: SettingWithCopyWarni

c1
c0 -- flow computers
c1 -- reportes
c2 -- meter streams
c3 -- measurement system
c4 -- equipments/equipment tags
c5 -- equipment types, equipment class
c6 -- variables
c7 -- calibraciones
c8 -- incerteza
c2
c0 -- flow computers
c1 -- reportes
c2 -- meter streams
c3 -- measurement system
c4 -- equipments/equipment tags
c5 -- equipment types, equipment class
c6 -- variables
c7 -- calibraciones
c8 -- incerteza


In [ ]:
# 1. Preprocesar los chunks en una lista de strings formateados
chunks_vals_lst = [
    f'{chunk["id"]} -- {chunk["topic"]}'
    for _, chunk in chunks_vals.iterrows()
]
print("Chunks disponibles:")
chunks_str = "\n".join(chunks_vals_lst)
print(chunks_str)
# 2. Crear una copia del DataFrame para no modificar el original
excel_queries_copy = excel_queries.copy()

# 3. Iterar sobre las preguntas en el DataFrame
for idx, row in excel_queries_copy.iterrows():
    question = row["question"]
    
    # 6. Solicitar al usuario que ingrese un chunk_id
    chunk_id = input(f"### Pregunta: {question} ###").strip()
    
    # 7. Si no se ingresa un chunk_id, salir del bucle
    if not chunk_id:
        print("Saliendo...")
        break
    
    # 8. Validar que el chunk_id sea un número
    if not chunk_id.isdigit():
        print("Error: El ID del chunk debe ser un número. Intenta de nuevo.")
        continue
    
    # 9. Asignar el chunk_id al DataFrame (formateado como "c{id}")
    chunk_id_formatted = f"c{chunk_id}"
    excel_queries_copy.at[idx, "chunk"] = chunk_id_formatted
    
    # 10. Confirmar la asignación
    print(f"Chunk asignado: {chunk_id_formatted}\n")


In [158]:
path = r"C:\Users\lauth\OneDrive\Desktop\LLM Assistant File - V2\fine_tuning\embeddings\sheets\datasets.xlsx"

chunks_vals = read_sheet(sheet_name="chunks", cols=["id", "content"], path=path)
excel_queries = read_sheet(
    sheet_name="sub_queries", cols=["question", "chunk", "text"], path=path
)




In [159]:
for index, item in excel_queries.iterrows():
    for i, chunk in chunks_vals.iterrows():
        if item["chunk"] == chunk["id"]:
            excel_queries.at[index, "text"] = chunk["content"]

C:\Users\lauth\AppData\Local\Temp\ipykernel_16892\2051594016.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'What is a measurement system?
A measurement system, also referred to as a delivery point, measurement point, or reception point, is used to measure and monitor fluids in industrial processes.

Key characteristics of a measurement system:
1. Measurement technology:
    - Differential: Used for precise measurements.
    - Linear: Used for straightforward measurements.

2. System identifier (TAG):
    - A unique identifier for the system.

3. Fluid type:
    - The system can measure gases, oils, condensates, water, steam, or other fluids.
4. System type:
    - Specifies the category or purpose of the system.

Measurement technology by fluid type:
- Gas measurement systems: Use both linear and differential measurement technologies.
- Oil measurement systems: Do not use linear or differential techn

In [161]:
p = r"C:\Users\lauth\OneDrive\Desktop\LLM Assistant File - V2\fine_tuning\embeddings\sheets\novo.xlsx"
excel_queries.to_excel(p, index=False)

# Trainer

In [ ]:
from datasets import load_dataset

# Cargar el dataset desde el Hub
dataset = load_dataset("Lauther/embeddings-train-semantic")

# Acceder a los conjuntos de entrenamiento y evaluación
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

In [ ]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator




In [181]:
%pip install sentence_transformers

  Obtaining dependency information for sentence_transformers from https://files.pythonhosted.org/packages/73/5d/8bb8486058f6cda9903588f452c4774f3c3bbba6f840c582c5e92ac9ab95/sentence_transformers-3.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for torch>=1.11.0 from https://files.pythonhosted.org/packages/0d/4a/e51420d46cfc90562e85af2fee912237c662ab31140ab179e49bd69401d6/torch-2.5.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/a1/a6/c5b78606743a1f28eae8f11973de6613a5ee87366796583fb74c67d54939/scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/af/25/caa430865749d504271757cafd24066d596217e83326155993980bc22f97/scipy-1.15.1-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.1 MB/


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
